In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

In [2]:
tf.__version__

'2.2.0'

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import OneHotEncoder

In [7]:
train_set = keras.datasets.fashion_mnist

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [10]:
# Нормализируем данные
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [11]:
X_train /= 255
X_test /= 255

In [12]:
enc = OneHotEncoder()

In [13]:
# Преобразуем метки в горячие векторы
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=42, test_size=0.1)

In [15]:
batch_size = 512
epochs=20

### Логистическая регрессия

In [16]:
from keras.callbacks import TensorBoard  
model_name = 'log_regression'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'./logs/{model_name}', histogram_freq=0, write_graph=True)

In [17]:
lr_model = tf.keras.models.Sequential()
lr_model.add(tf.keras.layers.Dense(num_classes, activation='softmax', input_shape=(784,)))

In [18]:
lr_model.compile(
    loss='categorical_crossentropy',
    optimizer='sgd',
    metrics=['accuracy']
)

In [22]:
hist = lr_model.fit(X_train,y_train,epochs = 1000,
          callbacks=[keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                         patience=15,
                                         restore_best_weights=True),
                     keras.callbacks.TensorBoard()
           ],
          validation_data=(X_test, y_test)
          )

Epoch 1/1000


ValueError: in user code:

    /usr/local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /usr/local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:531 train_step  **
        y_pred = self(x, training=True)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:886 __call__
        self.name)
    /usr/local/lib/python3.7/site-pac

In [ ]:
loss, accuracy = lr_model.evaluate(X_val, y_val, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

In [ ]:
%load_ext tensorboard

In [ ]:
import tensorflow as tf
import datetime, os

logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

### Полносвязная нейронка

In [ ]:
CNN_X_train = X_train.to_numpy()
CNN_X_val = X_val.to_numpy()
CNN_X_test = X_test.to_numpy()

CNN_X_train = CNN_X_train.reshape(X_train.shape[0], 28, 28, 1)
CNN_X_val = CNN_X_val.reshape(X_val.shape[0], 28, 28, 1)
CNN_X_test = CNN_X_test.reshape(X_test.shape[0], 28, 28, 1)

In [ ]:
# Модель для второго задания
model_name = 'CNN_1'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'./logs/{model_name}')

CNN_model = keras.models.Sequential()
CNN_model.add(keras.layers.Dense(32, activation='relu', input_shape=(784,)))
CNN_model.add(keras.layers.Dense(64, activation='relu'))
CNN_model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
CNN_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
                 )

In [ ]:
CNN_model.fit(
    CNN_X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(CNN_X_val , y_val),
    callbacks=[tensorboard]
)

In [ ]:
loss, accuracy = CNN_model.evaluate(CNN_X_val, y_val, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Вывод: при не очень большой разнице в времени обучения качество модели заметно лучше, тогда, возьмем для дальнейшего анализа

In [ ]:
import tensorflow as tf
import datetime, os

logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

### Полносвязная нейронка, но больше по размеру

In [ ]:
tf.__version__

In [ ]:
# Добавим еще 2 слоя (также добавим отступы, так как без него получена ошибка «Размер отрицательного размера ...»)
model_name = 'CNN_2'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'./logs/{model_name}')

CNN_model2 = tf.keras.models.Sequential()
CNN_model2.add(tf.keras.layers.Convolution2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(28,28,1)))
CNN_model2.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model2.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model2.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model2.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model2.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model2.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model2.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model2.add(tf.keras.layers.Flatten())
CNN_model2.add(tf.keras.layers.Dense(64, activation='relu'))
CNN_model2.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [ ]:
CNN_model2.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

CNN_model2.fit(
    CNN_X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(CNN_X_val , y_val),
    callbacks=[tensorboard]
)

In [ ]:
loss, accuracy = CNN_model2.evaluate(CNN_X_val, y_val, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

***Вывод***: если добавить еще слой и выкрутить количество нейронов, качество повысится, но очень незначительно. 
Early Stopping - хороший метод оптимизации, позволяет сэкономить ресурсы на обучении эпох, которые уже не повысят качество.

In [ ]:
# Добавим батч нормализацю после каждого слоя свертки
model_name = 'CNN_3'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'./logs/{model_name}')

CNN_model3 = tf.keras.models.Sequential()
CNN_model3.add(tf.keras.layers.Convolution2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(28,28,1)))
CNN_model3.add(tf.keras.layers.BatchNormalization())
CNN_model3.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model3.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model3.add(tf.keras.layers.BatchNormalization())
CNN_model3.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model3.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model3.add(tf.keras.layers.BatchNormalization())
CNN_model3.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model3.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model3.add(tf.keras.layers.BatchNormalization())
CNN_model3.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model3.add(tf.keras.layers.Flatten())
CNN_model3.add(tf.keras.layers.Dense(64, activation='relu'))
CNN_model3.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [ ]:
CNN_model3.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

CNN_model3.fit(
    CNN_X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(CNN_X_val , y_val),
    callbacks=[tensorboard]
)

In [ ]:
loss, accuracy = CNN_model3.evaluate(CNN_X_val, y_val, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

In [ ]:
# Добавим метод оптимизации drop-out после каждого максимальногоmax pooling слоя
model_name = 'CNN_4'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'./logs/{model_name}')

CNN_model4 = tf.keras.models.Sequential()
CNN_model4.add(tf.keras.layers.Convolution2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(28,28,1)))
CNN_model4.add(tf.keras.layers.BatchNormalization())
CNN_model4.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
CNN_model4.add(tf.keras.layers.Dropout(0.2))

CNN_model4.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model4.add(tf.keras.layers.BatchNormalization())
CNN_model4.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
CNN_model4.add(tf.keras.layers.Dropout(0.2))

CNN_model4.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model4.add(tf.keras.layers.BatchNormalization())
CNN_model4.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
CNN_model4.add(tf.keras.layers.Dropout(0.2))

CNN_model4.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model4.add(tf.keras.layers.BatchNormalization())
CNN_model4.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
CNN_model4.add(tf.keras.layers.Dropout(0.2))

CNN_model4.add(tf.keras.layers.Flatten())
CNN_model4.add(tf.keras.layers.Dense(64, activation='relu'))
CNN_model4.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [ ]:
CNN_model4.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

CNN_model4.fit(
    CNN_X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(CNN_X_val , y_val),
    callbacks=[tensorboard]
)

In [ ]:
loss, accuracy = CNN_model4.evaluate(CNN_X_val, y_val, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

In [ ]:
# Попробуем batch нормализацию полсе полностью связанных слоев нейронки
model_name = 'CNN_5'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'./logs/{model_name}')

CNN_model5 = tf.keras.models.Sequential()
CNN_model5.add(tf.keras.layers.Convolution2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(28,28,1)))
CNN_model5.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model5.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model5.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model5.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model5.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model5.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model5.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model5.add(tf.keras.layers.Flatten())
CNN_model5.add(tf.keras.layers.Dense(64, activation='relu'))
CNN_model5.add(tf.keras.layers.BatchNormalization())
CNN_model5.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [ ]:
CNN_model5.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

CNN_model5.fit(
    CNN_X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(CNN_X_val , y_val),
    callbacks=[tensorboard]
)

In [ ]:
loss, accuracy = CNN_model5.evaluate(CNN_X_val, y_val, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

***Вывод***: Свёрточная нейронная сеть не дает большого прироста качества по сравнению с полносвязной нейросетью (всего 0.01 пункт accuracy). Увеличение слоев и добавление batch normalization также дают совсем небольшой прирост качества.

## Выбор лучшей модели

Как показвают значения метрики accuracy на валидационных данных, лучшего результата достигла модель CNN_4 (наблюдаем на 9 эпохе обучения) - 0.92. Посмотрим, как будет вести себя эта модель на новых данных, посчитав метрику accuracy на тестовом сете.

In [ ]:
loss, accuracy = CNN_model4.evaluate(CNN_X_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

***Итоги***: CNN обучается медленнее всего, но обспечивает наибольшую производительность.